In [1]:
import os 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

In [2]:
driver_path= os.path.join(os.getcwd(),'chromedriver')
driver= webdriver.Chrome(driver_path)

In [3]:
query=input('Enter search query')
search=query.replace(" ","%20")
url = 'https://www.flipkart.com/search?q=' + search + '&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page='
my_url=url+'1'
print(my_url)

Enter search querydesktop pc
https://www.flipkart.com/search?q=desktop%20pc&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1


In [4]:
driver.get(my_url)

In [5]:
content = driver.page_source
soup = BeautifulSoup(content)

In [6]:
# if query == "mobile" or "mobiles"
    ##getting data of mobiles items and laptop
#spandiv= soup.find("div",attrs={'class':'_1AtVbE col-12-12'})
import requests

if (query == "mobile") or (query =="laptop"):
    print(query)
    span=soup.find('div', attrs={'class':'_2MImiq'})
    try:
        pages=int(span.text[10:13])
    except:
        pages=int(span.text[10:12])
    description=[]
    products=[] #List to store name of the product
    prices=[] #List to store price of the product
    ratings=[]#List to store rating of the product
    images=[]
    image_link=[]
    image_download=[]
    discount_price=[]
    discount_prices=[]

    description_txt=[]
    for i in range(1,15):
        URL=url+str(i)
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'html5lib')
        for a in soup.findAll('a',href=True, attrs={'class':'_1fQZEK'}):
                name=a.find('div', attrs={'class':'_4rR01T'})
                price=a.find('div', attrs={'class':'_30jeq3 _1_WHN1'})
                discount_price=a.find('div', attrs={'class':"_3I9_wc _27UcVY"})
                desc = a.find('ul', attrs={'class':'_1xgFaf'})
                desc1 = desc.find('li', attrs={'class':'rgWa7D'}).text
                desc1 =str(desc1)
                images = a.find('img', attrs={'class':'_396cs4 _3exPp9'})
                image_link = images["src"]
                products.append(name.text)
                prices.append(price.text)
                discount_prices.append(discount_price)
                description.append(desc1)
                image_download.append(image_link)
    print(len(description))
    print(len(products))
    print(type(image_download))

elif(query == "desktop pc"):
    print(query)
    span=soup.find('div', attrs={'class':'_2MImiq'})
    try:
        pages=int(span.text[10:13])
    except:
        pages=int(span.text[10:12])
    description=[]
    products=[] #List to store name of the product
    prices=[] #List to store price of the product
    ratings=[]#List to store rating of the product
    image_links=[]
    image_download=[]
    discount_price=[]
    discount_prices=[]
    for i in range(1,5):
        URL=url+str(i)
        print(URL)
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'html5lib')
        for a in soup.findAll('div', attrs={'class':'_13oc-S'}):
            name=a.find('a', attrs={'class':'s1Q9rs'})
            price=a.find('div', attrs={'class':'_30jeq3'})
            discount_price=a.find('div', attrs={'class':'_3I9_wc'})
           # rating=a.find('div', attrs={'class':'_3LWZlK'})
            desc = a.find('div', attrs={'class':'_3Djpdu'})
            #description = a.find('li', attrs={'class':'rgWa7D'})
            #description = description["li"]
            #hearder_link_tags = description.find_all("li")
            #des = description["li"]
            images = a.find('img', attrs={'class':'_396cs4 _3exPp9'})
            image_link = images["src"]
            products.append(name.text)
            prices.append(price.text)
            discount_prices.append(discount_price.text)
           # ratings.append(rating.text)
            description.append(desc.text)
            image_download.append(image_link)
    print(products)
        
else:
    print("Please Enter a valid query ")
    

desktop pc
https://www.flipkart.com/search?q=desktop%20pc&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1
https://www.flipkart.com/search?q=desktop%20pc&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=2
https://www.flipkart.com/search?q=desktop%20pc&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=3
https://www.flipkart.com/search?q=desktop%20pc&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=4
['Lenovo Ideacentre Celeron Dual Core (4 GB DDR4/1 TB/Fre...', 'Lenovo ideacentre Core i3 (10th Gen) (8 GB DDR4/512 GB ...', 'Lenovo Ideacentre Pentium Quad Core (4 GB DDR4/1 TB/Win...', 'ZOONIS CORE 2 DUE Core 2 Duo (4 GB DDR3/500 GB/120 GB S...', 'ASUS AIO M241DAK Athlon Dual Core (4 GB DDR4/1 TB/Windo...', 'DZAB All-in-one Core i3 (4 GB DDR3/500 GB/Windows 10 Ho...', 'Assembled 1ST RC-IT-SOLUTIONS Core 2 Duo (4 GB DDR2/320...', 'HP All in One PC Core i5 (9th Gen) (8 GB DDR4/1 T

In [7]:
df = pd.DataFrame({'Product Name':products,'Price':prices,'Image Link':image_link,'Desciption':description,'Discount':discount_prices})
df.to_csv('products_'+query+'.csv', index=False, encoding='utf-8')

In [8]:
import shutil, sys
#path = os.path.join(parent_dir, directory)
try:
    os.mkdir("Flipkart_images/"+query)
    for idx, links in enumerate(image_download):
    #for links in range(1,image_download):
        filename=links.split('-')[-3]
        r = requests.get(image_download[idx], stream=True) #Get request on full_url
        if r.status_code == 200:                     #200 status code = OK
           with open("Flipkart_images/"+query+"/{}.png".format(filename+str(idx)), 'wb') as f: 
              r.raw.decode_content = True
              shutil.copyfileobj(r.raw, f)
except:
    for idx, links in enumerate(image_download):
    #for links in range(1,image_download):
        filename=links.split('-')[-3]
        r = requests.get(image_download[idx], stream=True) #Get request on full_url
        if r.status_code == 200:                     #200 status code = OK
           with open("Flipkart_images/"+query+"/{}.png".format(filename+str(idx)), 'wb') as f: 
              r.raw.decode_content = True
              shutil.copyfileobj(r.raw, f)

In [23]:
filename

'narzo-30-5g-rmx3242-realme-original-imag45yjdkgxddvh.jpeg?q=70'

In [26]:
links.split('-')[-3]

'realme'